<a href="https://colab.research.google.com/github/Mathildeholst/Git_training/blob/main/F5_Workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Fine-tuning your model using KerasTuner


In [3]:
!pip install keras-tuner -q
import keras_tuner as kt
from tensorflow import keras
from tensorflow.keras import layers


def build_model(hp):
    units = hp.Int(name="units", min_value=16, max_value=64, step=16)
    model = keras.Sequential([
        layers.Dense(units, activation="relu"),
        layers.Dense(10, activation="softmax")
    ])
    optimizer = hp.Choice(name="optimizer", values=["rmsprop", "adam"])
    model.compile(
        optimizer=optimizer,
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"])
    return model

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.4 MB/s eta 0:00:00


What does this KerasTuner code do?
* You're building a model where you don't hardcode the hyperparameters.

Instead, you let KerasTuner test different options automatically.

* hp.Int: tries different values for units (16, 32, 48, 64)

* hp.Choice: lets KerasTuner pick an optimizer (either "rmsprop" or "adam")


You can also use:

* hp.Float: try decimal values (e.g., learning rate)

* hp.Boolean: choose between True or False

This helps you find the best model configuration without guessing everything yourself.

In [4]:
tuner = kt.BayesianOptimization(
    build_model,
    objective="val_accuracy",
    max_trials=100,
    executions_per_trial=2,
    directory="mnist_kt_test",
    overwrite=True,
)

objective="val_accuracy",
* Specify the metric that the tuner will seek to optimize. Always specify validation metrics, since the goal of the search process is to find models that generalize!

max_trials=100,
* Maximum number of different model configurations (“trials”) to try before ending the search.

executions_per_trial=2,
 * executions_per_trial trains the same model setup multiple times and averages the results, so you get more reliable scores.

directory="mnist_kt_test",
* Where to store search logs

overwrite=True,
* overwrite=True means KerasTuner will delete old search results and start fresh. Use this if you’ve changed your model setup. Set it to False if you want to continue tuning from where you left off.












In [ ]:
callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=5),
]
tuner.search(
    x_train, y_train,
    batch_size=128,
    epochs=100,
    validation_data=(x_val, y_val),
    callbacks=callbacks,
    verbose=2,
)

This code uses KerasTuner to find the best model setup.

You train for up to 100 epochs, but:

* You don’t know how many epochs are actually needed.

* So you use EarlyStopping: it watches the validation loss (val_loss) and stops training if it doesn't improve for 5 rounds (patience=5).

This helps avoid overfitting and saves time.

verbose=2
* verbose=2 is a clean and readable way to follow training progress








In [ ]:
top_n = 4
best_hps = tuner.get_best_hyperparameters(top_n)

* This gives you the top 4 best hyperparameter sets found during tuning.
* You can then use one of them to build and train your final model.

In [ ]:
def get_best_epoch(hp):
    model = build_model(hp)
    callbacks=[
        keras.callbacks.EarlyStopping(
            monitor="val_loss", mode="min", patience=10)
    ]
    history = model.fit(
        x_train, y_train,
        validation_data=(x_val, y_val),
        epochs=100,
        batch_size=128,
        callbacks=callbacks)
    val_loss_per_epoch = history.history["val_loss"]
    best_epoch = val_loss_per_epoch.index(min(val_loss_per_epoch)) + 1 # Python uses zero-based indexing, so we add 1 to get human-readable epoch number
    print(f"Best epoch: {best_epoch}")
    return best_epoch

This function helps you find the best number of epochs to train your model, using early stopping.

Trains it for up to 100 epochs, but stops early if validation loss doesn't improve for 10 epochs.

You want to know how many epochs to train your final model without overfitting. This helps you train just enough — not too much, not too little.

In [ ]:
def get_best_trained_model(hp):
    best_epoch = get_best_epoch(hp)
    model = build_model(hp)
    model.fit(
        x_train_full, y_train_full,
        batch_size=128,
        epochs=int(best_epoch * 1.2))
    return model

This function trains the final model using the best number of epochs found earlier.

Trains it on all the training data (x_train_full, y_train_full) — including what was previously used for validation.

Trains for a bit longer than before (about 20% more epochs) to make the most of the full dataset.



In [ ]:
best_models = []
for hp in best_hps:
    model = get_best_trained_model(hp)
    model.evaluate(x_test, y_test)
    best_models.append(model)

This code trains and evaluates the best models found by KerasTuner.